In [1]:
# Setup
import matplotlib as plt
import numpy as np
from dolo import *
import dolark

In [11]:
filename = 'Individual.yaml'
pcat(filename)

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56


In [12]:
#yaml_import(filename) reads the YAML file and generates a model object.
model = yaml_import(filename)

In [13]:
#check whether steady-state variables are correct by computing the model equations at the steady state.
model.residuals()

{'transition': array([0.]), 'arbitrage': array([0.00112833])}

In [14]:
#have a look at all the model equations and check that all residual errors are 0 at the steady-state, but with less display prescision.
print(model)


        Model:
        ------
        name: "Agent"
        type: "dtcc"
        file: "Individual.yaml

Equations:
----------

transition
 1  : 0.0000 : a(0) == i(-(1))

arbitrage
 1  : 0.0011 : (1) - (((((beta) * (1 + r(1))) ** ((1) / (RiskAversion))) * (c(0))) / (c(1)))

definitions
 1  : c = (1+r)*a + w*exp(e) - i




In [18]:
model.exogenous

Exception: Unknown exogenous type None.

In [16]:
dr_pert = approximate_controls(model)

Exception: Unknown exogenous type None.

In [17]:
dr_global = time_iteration(model)

Exception: Unknown exogenous type None.